In [1]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

### loading the test_df as generator

In [3]:
from ebcache import stackPONmatrix
from functools import partial
from multiprocessing import Pool

In [82]:
config = dict(
    ZDfactor=13,
    threads=8,
    pon_path=pon_path,
    
    zero_path=os.path.join(pon_path, "zero"),
    zerostring = "0|0|0|0|0",
    min_zt = 10000, # minimum lenght of zeroT_df to bother zerocache,
    chunksize=10000
)
chr19_gen = pd.read_csv(os.path.join(pon_path, "matrix/chr19.pon.gz"), nrows=100000, chunksize=10000, compression="gzip", sep="\t")

stack_pool = Pool(config['threads'])

dfs = stack_pool.imap(partial(stackPONmatrix, config), chr19_gen)


In [83]:
%time test = pd.concat(dfs).reset_index(drop=True)
test

CPU times: user 1.25 s, sys: 508 ms, total: 1.76 s
Wall time: 15.7 s


,Chr,Start,Ref,Alt,strand,T,D
0,chr19,66218,C,A,L,0|0|0|0|0,0|0|0|0|5
1,chr19,66218,C,A,R,0|0|0|0|0,0|0|0|0|0
2,chr19,66218,C,G,L,0|0|0|0|0,0|0|0|0|5
3,chr19,66218,C,G,R,0|0|0|0|0,0|0|0|0|0
4,chr19,66218,C,C,L,0|0|0|0|0,0|0|0|0|5
...,...,...,...,...,...,...,...
1199995,chr19,1010759,T,T,R,0|0|0|0|0,40|74|109|118|118
1199996,chr19,1010759,T,I,L,0|0|0|0|0,69|109|137|137|137
1199997,chr19,1010759,T,I,R,0|0|0|0|0,40|74|109|118|118
1199998,chr19,1010759,T,D,L,0|0|0|0|0,69|109|137|137|137


### load one zero_file and update

In [84]:
from zerocache import load_zero_list
from script_utils import show_output

def zero2AB(stack_df, config):
    '''
    go through zero files and get matching ABs from zero cache for tumor == zerostring
    '''

    # split off the zeroT_df
    zeroT_df = stack_df.loc[stack_df['T'] == config['zerostring'], :].sort_values("D")
    # if zeroT_df is too small, just return stack_df and empty AB_df
    if len(zeroT_df) < config['min_zt']:
        return stack_df, pd.DataFrame()
    
    stack_df = stack_df.loc[stack_df['T'] != config['zerostring'], :]
    
    # load the zerofiles and merge incrementally to save memory
    AB_dfs = []
    for zfile in load_zero_list(config['zero_path'], 5)[0]:
        try:
            zgen = pd.read_csv(zfile, sep="\t", compression="gzip", chunksize=config['chunksize'])
            show_output(f"Loading {zfile}")
            for zdf in zgen:
                merge_df = zeroT_df.merge(zdf, how="left")
                AB_df = merge_df.query("AB == AB")
                AB_dfs.append(AB_df)
                zeroT_df = merge_df.query("AB != AB").drop("AB", axis=1)
        except Exception as e:
            show_output(f"{zfile} could not be loaded, {e}", color="warning")
    
    AB_df = pd.concat(AB_dfs)
    
    # load remaining
    stack_df = pd.concat([stack_df, zeroT_df]).reset_index(drop=True)
    show_output(
            f"{len(AB_df.index)} matching lines found in zero cache! Computing remaining {len(stack_df.index)} lines.",
            multi=False
        )
    return stack_df, AB_df

In [85]:
stack_df, AB_df = zero2AB(test, config)


12:31:29 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.0.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.1.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.2.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.3.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.4.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.5.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.6.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.7.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.8.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.9.gz
12:31:52 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.10.gz
12:31:52 : Loading /Users/mahtin/Dropbox/I

12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.78.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.79.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.80.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.81.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.82.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.83.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.84.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.85.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.86.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.87.gz
12:31:53 : Loading /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zero5.88.gz
12:31:53 : Loading /Users/mahtin

In [86]:
stack_df

,Chr,Start,Ref,Alt,strand,T,D
0,chr19,71000,G,D,L,1|0|0|0|0,14|14|6|10|5
1,chr19,71049,C,A,L,1|0|0|0|0,22|34|11|18|9
2,chr19,71052,T,C,L,1|0|0|0|0,23|33|12|19|9
3,chr19,71058,C,A,L,0|0|0|1|0,23|33|12|18|9
4,chr19,71062,C,A,L,0|0|0|1|0,23|34|12|18|9
...,...,...,...,...,...,...,...
33508,chr19,1010755,C,A,L,1|0|0|1|0,108|139|144|139|76
33509,chr19,1010756,G,A,R,0|0|0|1|0,76|122|124|126|37
33510,chr19,1010756,G,T,L,0|0|0|1|0,110|142|146|142|78
33511,chr19,1010756,G,T,R,0|0|0|0|1,76|122|124|126|37


## computation has  to be done per PON string
+ allows use of same function for both PONcache and matrix2AB
+ allows optimal caching of the results

In [ ]:
from ebcache import PONmatrix2AB_multi
from zerocache import collapse_zeros

# _ = collapse_zeros(os.path.join(pon_path, "zero"), ponsize=5)

PONAB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunksize":500000,
    "pon_path": pon_path,
    "ZDfactor": 13 # how much complexity remains after flattening the tumor-zero lines
}

chr7_AB_df = PONmatrix2AB_multi(chr7_df, config=PONAB_config)
chr7_AB_df

In [ ]:
chr7_AB_df.to_csv(os.path.join(pon_path, "ABcache/chr7.AB"), sep="\t", index=False)

In [ ]:
from zerocache import collapse_zeros
zdf = collapse_zeros(os.path.join(pon_path, "zero"), ponsize=5, ZDfactor=13)
zdf

### test collapse_zeros

In [ ]:
from zerocache import flatten_zeros
from script_utils import show_output

def collapse_zeros(zero_path, ZDfactor=13):
    """
    reduces all zero_files to zero.0.csv
    """
    # get all the zero files
    show_output(f"Collapsing all zero files in {zero_path} into zero.0.csv")
    zero_files = [
        os.path.join(zero_path, file
                     
                     
                     
                     
                     
                     
                     
                     
                     **********************'''''''''''''
        for file in os.listdir(zero_path)
        if os.path.isfile(os.path.join(zero_path, file)) and "zero" in file
    ]
    zero_df = pd.DataFrame()
    if len(zero_files) == 0:
        show_output(f"No zerofiles found in {zero_path}!")
        return
    for file in zero_files:
        show_output(f"Cleaning up zero file {file}", time=False)
        try:
            zdf = pd.read_csv(file, sep="\t")
            # concat to zdf_all and drop duplicates
            zero_df = pd.concat([zero_df, zdf]).drop_duplicates("D")
            # reduce complexity using condense factor
            zero_df.loc[:, "D"] = flatten_zeros(
                zero_df["D"], ZDfactor=ZDfactor
            )
        except:
            show_output(f"{file} could not be loaded", color="warning", time=False)
        os.remove(file)

    zero_df = zero_df.sort_values("D")

    zero_df.to_csv(os.path.join(zero_path, "zero.0.csv"), sep="\t", index=False)
    show_output(f"Written collapsed zerofile ({len(zero_df.index)} lines) to {zero_path}/zero.0.csv")
    return zero_df

In [ ]:
zdf = collapse_zeros(os.path.join(pon_path, "zero"), ZDfactor=13)